# IMPORTS

In [1]:
import os
from window import window
from cv2 import imread, imwrite

# Imports for Neural Network
from ml.photo.optimize import optimize
from ml.photo.evaluate import ffwd, ffwd_to_img, ffwd_different_dimensions
from ml.photo.utils import isimage
from tensorflow.logging import set_verbosity, ERROR, DEBUG

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

set_verbosity(DEBUG)

# WORKER

In [2]:
# TODO: Move to file
class Photo:
    def __init__(self):
        self.dataSetDir = 'D:\ML\datasets\COCO2014'
        self.imageDir = 'photo\style{}'
        self.styleDir = 'photo\style{}.jpg'
        self.examplePaths = ['photo\example{}.jpg'.format(i) for i in range(3)]
        self.modelDir = 'D:\ML\models\smart_paint\style{}'
        self.vggPath = 'D:\ML\models\imagenet-vgg-verydeep-19.mat'

        self.batch = 22
        self.epochs = 15
        self.device = '/gpu:1'
        self.learningRate = 0.001
        self.contentWeight = 7.5
        self.styleWeight = 200
        self.tvWeight = 200
        self.checkIterations = 500

    def train(self, styleCount):
        modelDir = self.modelDir.format(styleCount)
        imagesDir = self.imageDir.format(styleCount)
        imwrite(imagesDir + '/c{}_s{}_tv{}.jpg'.format(self.contentWeight,  self.styleWeight, self.tvWeight), style)
        
        print('Training {} has been began\n'.format(styleCount))
        
        for preds, losses, iteration, epoch in optimize(self.dataSetDir, style, self.epochs, self.batch,
                                                    self.contentWeight, self.styleWeight, self.tvWeight,
                                                    self.vggPath, modelDir, modelDir + "/e0_i4137/model.ckpt", self.checkIterations, 
                                                    self.learningRate, self.device):
            styleLoss, contentLoss, tvLoss, loss = losses
            print('\nEpoch %d, Iteration: %d, Loss: %s' % (epoch, iteration, loss))
            print('Style: %s, Content:%s, Tv: %s\n' % (styleLoss, contentLoss, tvLoss))
            
            files = [imagesDir + '/e{}_i{}-{}.jpg'.format(epoch, iteration, image) for image in range(len(self.examplePaths))]
            ffwd_different_dimensions(self.examplePaths, files, modelDir + '/e{}_i{}'.format(epoch, iteration), '/gpu:0')
            
        print('\nTraining {} has been completed\n\n\n'.format(styleCount))

    def work(self, imagePath, styleID): # TODO: me
        checkpointPath = self.modelDir + '/style%s' % styleID  + '/e0_i3761/model.ckpt'
        if type(imagePath) is str:
          imagePath = [imagePath]
        return ffwd(imagePath, 'example.jpg', checkpointPath, self.device, self.batch)
      
    def autowork(self, style):
      files = ['example{}.jpg'.format(image) for image in range(len(self.examplePaths))]
      ffwd_different_dimensions(self.examplePaths, files, self.modelsDir + '/style%s/model.ckpt' % style)

# MAIN CODE

## Initializate vatiables

In [3]:
photo = Photo()

## Train

In [ ]:
%%time
%autosave 30
photo.train(6)

Autosaving every 30 seconds
Training 6 has been began

Train set has been trimmed slightly..
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
